In [31]:
from pdfminer import pdfpage
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
import nltk
from nltk import sent_tokenize, word_tokenize
import pandas as pd
from io import StringIO
import csv

from tqdm import tqdm
from difflib import SequenceMatcher
import re
import pickle
import re

In [367]:

def pdf2token(pdf,name):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams(char_margin=30, line_margin=2, boxes_flow=1)
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    caching = True
    pagenos = set()
    fp = open(pdf, 'rb')
    location = r'C:/Users/33669/PycharmProjects/OCR/trainer/ner_data/training/'+name+'.csv'
    writer = csv.writer(open(location, 'w', newline=''))

    for pagenumber, page in enumerate(pdfpage.PDFPage.get_pages(fp, check_extractable=True)):
        #print(pagenumber)
        if pagenumber:
            interpreter.process_page(page)
            data = retstr.getvalue()
            #print('PAGE DATA +++++++++++++' + data)
            sent_text = sent_tokenize(data)
            for sent in sent_text:
                #print('pre: '+sent)
                cleansent = clean(sent)
                encoded_string = cleansent.encode("ascii", "ignore")
                cleansent = encoded_string.decode()
                cleansent = " ".join(cleansent.split())
                #print('post: '+cleansent)
                writer.writerow([cleansent.lower()])
                #writer.writerow('\n')

def txttrain(train):
    entpair = []
    eg = []
    ent = []
    print('Runnning text trian')
    with open(train, 'r') as f:
        for line in f:
            text = line.rstrip('\n')
            junk = ["!", "#", "$", "%", "/", "*", ":", ";", "<", "=", ">", "?", "@",
               "\\", "_", "`", "~",".","(",")"]
            defi = []
            defi = text.lower().split(',')
            if len(defi)>=3:
                for w in range(1,len(defi)-1):
                    defi[1] = defi[1]+''+defi[w+1]
            defi[1] = defi[1].strip('\n "()')
            for i in defi[1]:
                if i in junk:
                    defi[1] = defi[1].replace(i, ' ')
            defi[1] = " ".join(defi[1].split())
            temp = defi[1]
            defi[1] = defi[0]
            defi[0] = temp
            print('[ '+defi[0]+' , '+defi[1]+' ]')
            eg.append(defi[0])
            ent.append(defi[1])
        entpair = list(zip(eg,ent))
    return(entpair)
            

    
def matcher(string, pattern):
    '''
    Return the start and end index of any pattern present in the text.
    '''
    match_list = []
    pattern = pattern.strip("[]'")
    string = string.strip("[]'")
    match = []
    '''
    i=0
    x = 0
    while x!= -1:
      x = string.find(pattern, i)
      if x!= -1:           
        tup = (x, x+len(pattern))
        #print('Value: '+pattern+'Location:    '+str(tup)+'\n\n'+string)
        match.append(tup)
        i = x+len(pattern)
    #match.pop()
    return match, string
    '''
    seqMatch = SequenceMatcher(None, string, pattern, autojunk=False)
    match = seqMatch.find_longest_match(0, len(string), 0, len(pattern))
    if (match.size == len(pattern)):
        start = match.a
        end = match.a + match.size
        match_tup = (start, end)
        string = string.replace(pattern, "X" * len(pattern), 1)
        print(string)
        match_list.append(match_tup)
    return match_list, string
    
        
    

def mark_sentence(s, match_list):
    '''
    Marks all the entities in the sentence as per the BIO scheme. 
    '''
    s = s.strip("[]'")
    print(s)
    
    word_dict = []
    first = 0    
    itr = 0
    
    for multiple in match_list:
        beg = False 
        print('current itteration- '+str(multiple))
        start = multiple[0]
        end = multiple[1]
        e_type = multiple[2]
        temp_str = s[start:end]
        tmp_list = word_tokenize(temp_str)
        if itr!=(len(match_list)-1):
            substring = s[first:end]
            print('Itteration: '+str(itr)+str(substring))
        else:
            substring = s[first:]
            print('Itteration: '+str(itr)+str(substring))
            
        print('First Identifier:    '+tmp_list[0])    
        for word in word_tokenize(substring):
            x = word.find(tmp_list[0])
            if x!=0:
                word_dict.append(word+' '+'O')
            if x==0:    
                word_dict.append(word+' '+'B-' + e_type)
                beg = True
                print('Begiening detected : '+word)
                
            if len(tmp_list) > 1 and beg is True:
                for w in tmp_list[1:]:
                    t = word.find(w)
                    if t==0:
                        print(str(t)+' Inner detected : '+word+' idf> '+w)
                        word_dict.pop()
                        word_dict.append(word+' '+'I-' + e_type)
        first=end
        itr=itr+1
    print('Final Word outcome:::    '+str(word_dict)+'\n\n\n')
    return word_dict

def clean(text):
    '''
    Just a helper fuction to add a space before the punctuations for better tokenization
    '''
    filters = ["!", "#", "$", "%", "(", ")", "/", "*", ":", ";", "<", "=", ">", "?", "@", "[",
               "\\", "]", "_", "`", "{", "}", "~"]
    text = text.strip('\n ')
    text = text.replace('\n', '').replace('\r', '')
    for i in text:
        i = i.replace("'", '').replace("`", '').replace(".",'').replace("°",'C')
        if i in filters:
            text = text.replace(i, " "+i+' ')
    return text

def create_data(df, text, filepath):
    '''
    The function responsible for the creation of data in the said format.
    1234
    1>1,a>
    '''

    with open(filepath , 'w', newline='') as f:
        for sentence in text:
            text_ = sentence       
            match_list = []
            for i,y in df:
                a, text_ = matcher(str(sentence), i)
                if a:
                    #print(i)
                    #print(str(a)+str(text_))
                    for itr in a:
                        print((itr[0], itr[1], y))
                        match_list.append((itr[0], itr[1], y))
            if match_list:
                print('Final match list:    '+str(match_list))
                d = mark_sentence(str(sentence), match_list)
                for i in d:
                    f.writelines(str(i)+'\n')
                f.writelines('\n')
            


In [373]:
def main():
   
    filename = 'TSLT'
    ## path to save the txt file.
    pdf = r'C:/Users/33669/PycharmProjects/OCR/trainer/ner_data/raw/'+filename +'.pdf'
    filepath = r'C:/Users/33669/PycharmProjects/OCR/trainer/ner_data/raw/'+filename +'.txt'
    sentoken = r'C:/Users/33669/PycharmProjects/OCR/trainer/ner_data/training/'+filename +'.csv'
    final = r'C:/Users/33669/PycharmProjects/OCR/trainer/ner_data/training/'+filename +'_final.txt'
    ## creating the file.
    sentences = []
    pdf2token(pdf,filename)
    data = txttrain(filepath) #Changed Drum info in txt file revert it back    
    with open(sentoken, 'r') as read_obj:
        csv_reader = csv.reader(read_obj)
        for row in csv_reader:
            sentences.append(row)            
    create_data(data, sentences, final)

if __name__ == '__main__':
    main()
    

TEXT REDUCTION FOR GIT UPLOAD: IGNORE ERROR'